# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
path = '../WeatherPy/output_data/cities.csv'
cities = pd.read_csv(path)
cities.drop('Unnamed: 0', axis='columns', inplace=True)
#some countries have nans, dropping them 
cities.dropna(inplace=True)
cities.head()

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Data
0,nikolskoye,59.70,30.79,44.60,70.0,75.0,6.71,RU,1619163102
1,punta arenas,-53.15,-70.92,44.60,100.0,90.0,3.44,CL,1619163103
2,olafsvik,64.89,-23.71,42.17,91.0,100.0,16.40,IS,1619163103
3,valdivia,-39.81,-73.25,50.00,100.0,90.0,2.30,CL,1619163104
4,yellowknife,62.46,-114.35,12.20,62.0,90.0,4.61,CA,1619163104


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

locations = cities[['Lat', 'Lng']]
weight = cities.Humidity

In [4]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weight, dissipating=False)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#weather between 70 and 80
hotel_df = cities[(cities.Max_Temp > 70) & (cities.Max_Temp < 80)]

#wind speed less than 10
hotel_df = hotel_df[hotel_df.Wind_Speed < 10]

#zero cloudiness
hotel_df = hotel_df[hotel_df.Cloudiness < 10]

hotel_df.head()

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Data
42,vredendal,-31.67,18.50,71.80,46.0,0.0,1.97,ZA,1619163125
91,plettenberg bay,-34.05,23.37,74.86,55.0,2.0,4.41,ZA,1619163156
126,miramar,25.99,-80.23,75.20,68.0,1.0,6.91,US,1619163177
215,boa vista,2.82,-60.67,77.00,83.0,0.0,9.22,BR,1619163232
309,tsiroanomandidy,-18.77,46.03,79.20,54.0,2.0,7.74,MG,1619163288


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df['Hotel_Name'] = ""
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

for index, row in hotel_df.iterrows():
    params = {
        'location':f"{row['Lat']}, {row['Lng']}",
        'radius':5000,
        'keyword':'hotel',
        'key':g_key
    }
    response = requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index, 'Hotel_Name'] = response['results'][0]['name']
    except:
        print(f"Skipping {row['City']}... No hotel found")

#get rid of places where we could not locate a hotel
hotel_df.dropna(inplace = True)
hotel_df.head()

Skipping kruisfontein... No hotel found


,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Data,Hotel_Name
42,vredendal,-31.67,18.50,71.80,46.0,0.0,1.97,ZA,1619163125,River Lodge
91,plettenberg bay,-34.05,23.37,74.86,55.0,2.0,4.41,ZA,1619163156,The Plettenberg Hotel
126,miramar,25.99,-80.23,75.20,68.0,1.0,6.91,US,1619163177,Stadium Hotel
215,boa vista,2.82,-60.67,77.00,83.0,0.0,9.22,BR,1619163232,ibis Styles Boa Vista
309,tsiroanomandidy,-18.77,46.03,79.20,54.0,2.0,7.74,MG,1619163288,Agence BNI Madagascar


In [7]:
# NOTE: Do not change any of the code in this cell
#I gave hotel name an underscore

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))